## Realtime face recognition Code and weights are mainly from <a href="https://github.com/davidsandberg/facenet">David Sandberg GitHub</a>
<br>

<table align="left">
  <thead>
    <tr>
      <th><p style="text-align:center;">Environments</p></th>
      <th><p style="text-align:center;">Frameworks</th>
      <th><p style="text-align:center;">Mian Packages</th>
      <th><p style="text-align:center;">Notes</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><p style="text-align:center;">Python 3.7.7</td>
      <td><p style="text-align:center;">Tensorflow 1.15</td>
      <td><p style="text-align:center;">numpy, etc.,</td>
      <td><p style="text-align:center;">Suggest to creat you env through Anacond etc.,<br>due to the automatic installation of lots of packages<br>建議採用Anaconda建立環境, 可節省諸多package安裝過程</td>
    </tr>
    <tr>
      <td><p style="text-align:center;"> </td>
      <td><p style="text-align:center;"> </td>
      <td><p style="text-align:center;">OpenCV '4.4.0'</td>
      <td><p style="text-align:center;">Suggest to install the package by .whl file<br>建議下載whl檔案安裝, 透過Anaconda pip安裝的版本無法正常結束影片</td>
    </tr>
      <tr>
      <td><p style="text-align:center;"> </td>
      <td><p style="text-align:center;"> </td>
      <td><p style="text-align:center;"> </td>
      <td><p style="text-align:center;">we will use pretrained weights from David to run the model<br>我們會用David Sandberg已經學習好的參數(det1/det2/det3)來跑模型</td>
    </tr>
  </tbody>
</table>

In [1]:
# MIT License
# 
# Copyright (c) 2016 David Sandberg
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to dealr
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER 
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six import string_types, iteritems

import numpy as np
import tensorflow as tf
#from math import floor
import cv2
import os
import matplotlib.pyplot as plt

from scipy import misc                # scipy.misc.imsave has been deprecated in newer Scipy versions.
import imageio                        # try this if you got error on 'scipy.misc' has no attribute 'imread'

import sys
import random
from time import sleep
import subprocess                    # Popen in store_revision_info() will need it
from skimage.transform import resize  # try this if you got error on 'scipy.misc' has no attribute 'imresize', scipy.misc.imresize is deprecated

C:\Users\user\anaconda3\envs\py37_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\envs\py37_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\envs\py37_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\envs\py37_tf\lib\site-packages\tensorflow\pyth

In [2]:
def layer(op):
    """Decorator for composable network layers."""

    def layer_decorated(self, *args, **kwargs):
        # Automatically set a name if not provided.
        name = kwargs.setdefault('name', self.get_unique_name(op.__name__)) # if name is not given, name = get_unique_name(op.__name__)
        # Figure out the layer inputs.
        if len(self.terminals) == 0:
            raise RuntimeError('No input variables found for layer %s.' % name)
        elif len(self.terminals) == 1:
            layer_input = self.terminals[0]
        else:
            layer_input = list(self.terminals)
        # Perform the operation and get the output.
        layer_output = op(self, layer_input, *args, **kwargs)  # layer_output = layerfunction(self, layer_input, *args, **kwargs)
        # Add to layer LUT.                                    # e.g. layer_output = prelu(self, inp)
        self.layers[name] = layer_output
        # This output is now the input for the next layer.
        self.feed(layer_output)
        # Return self for chained calls.
        return self

    return layer_decorated

In [3]:
class Network(object):

    def __init__(self, inputs, trainable=True):   # constructor : define object in network class, and attribute(inputs, trainable)
        # The input nodes for this network
        self.inputs = inputs
        # The current list of terminal nodes
        self.terminals = []
        # Mapping from layer names to layers
        self.layers = dict(inputs)
        # If true, the resulting variables are set as trainable
        self.trainable = trainable

        self.setup()

    def setup(self):  # Method
        """Construct the network. """
        raise NotImplementedError('Must be implemented by the subclass.')

    def load(self, data_path, session, ignore_missing=False):   # e.g. pnet.load(path, sess) in "def create_mtcnn" below
        """Load network weights.
        """
        data_dict = np.load(data_path, encoding='latin1').item() #pylint: disable=no-member

        for op_name in data_dict:
            with tf.variable_scope(op_name, reuse=True):
                for param_name, data in iteritems(data_dict[op_name]):
                    try:
                        var = tf.get_variable(param_name)
                        session.run(var.assign(data))
                    except ValueError:
                        if not ignore_missing:
                            raise

    def feed(self, *args):
        """Set the input(s) for the next operation by replacing the terminal nodes.
        The arguments can be either layer names or the actual layers.
        """
        assert len(args) != 0
        self.terminals = []
        for fed_layer in args:
            if isinstance(fed_layer, string_types):
                try:
                    fed_layer = self.layers[fed_layer]
                except KeyError:
                    raise KeyError('Unknown layer name fed: %s' % fed_layer)
            self.terminals.append(fed_layer)
        return self

    def get_output(self):
        """Returns the current network output."""
        return self.terminals[-1]

    def get_unique_name(self, prefix):
        """Returns an index-suffixed unique name for the given prefix.
        This is used for auto-generating layer names based on the type-prefix.
        """
        ident = sum(t.startswith(prefix) for t, _ in self.layers.items()) + 1
        return '%s_%d' % (prefix, ident)     # e.g return conv3 for 3rd conv in all the layers

    def make_var(self, name, shape):
        """Creates a new TensorFlow variable."""
        return tf.get_variable(name, shape, trainable=self.trainable)

    def validate_padding(self, padding):
        """Verifies that the padding is one of the supported ones."""
        assert padding in ('SAME', 'VALID')

    @layer
    def conv(self,
             inp,
             k_h,     # kernel size_h-direct.
             k_w,     # kernel size_w-direct.
             c_o,     # filters (channel_outter)
             s_h,     # stride_h-direct.
             s_w,     # stride_w-direct.
             name,                                            # define op_name here, then you can fine the weights in after
             relu=True,
             padding='SAME',                                  # notice here : the default "SAME" padding
             group=1,
             biased=True):
        self.validate_padding(padding)                        # Verify that the padding is acceptable
        c_i = int(inp.get_shape()[-1])                        # Get the number of channels in the input (channel_inner)
        # Verify that the grouping parameter is valid
        assert c_i % group == 0                              # % 1 == 0, assert value c_i is able to work
        assert c_o % group == 0                              # % 1 == 0, assert value c_o is able to work
        
        # Convolution for a given input and kernel
        convolve = lambda i, k: tf.nn.conv2d(input = i, filter = k, strides = [1, s_h, s_w, 1], padding=padding)  
        with tf.variable_scope(name) as scope:
            kernel = self.make_var('weights', shape=[k_h, k_w, c_i // group, c_o])       # define parameter_name here
            # This is the common-case. Convolve the input without any further complications.
            output = convolve(inp, kernel)                                               # define w*a
            # Add the biases
            if biased:
                biases = self.make_var('biases', [c_o])                                  # define parameter_name here
                output = tf.nn.bias_add(output, biases)                                  # define w*a + b
            if relu:
                # ReLU non-linearity
                output = tf.nn.relu(output, name=scope.name)                             # define activation (w*a + b)
            return output

    @layer
    def prelu(self, inp, name):
        with tf.variable_scope(name):
            i = int(inp.get_shape()[-1])
            alpha = self.make_var('alpha', shape=(i,))                                   # define prelu parameter_name here
            output = tf.nn.relu(inp) + tf.multiply(alpha, -tf.nn.relu(-inp))             # define prelu = ReLu(a) - alpha*ReLu(-a)
        return output
    # MTCNN conduct the nonlinearity by PReLU, 使用 Parametric ReLU (PReLU) 導入 nonlinearity activation
    # alpha is a learnable variable 是需要學習的參數
    
    @layer
    def max_pool(self, inp, k_h, k_w, s_h, s_w, name, padding='SAME'):                  # notice here : the default "SAME" padding
        self.validate_padding(padding)
        return tf.nn.max_pool(value = inp,
                              ksize=[1, k_h, k_w, 1],
                              strides=[1, s_h, s_w, 1],
                              padding=padding,
                              name=name)

    @layer
    def fc(self, inp, num_out, name, relu=True):
        with tf.variable_scope(name):
            input_shape = inp.get_shape()                        # get input shape, e.g. RNet fc input is [None, 3, 3, 64] [576, 128]
            if input_shape.ndims == 4:
                # The input is spatial. Vectorize it first.
                dim = 1
                for d in input_shape[1:].as_list():             # e.g. d = 3, 3, 64
                    dim *= int(d)                               # e.g. dim = 3 * 3 * 64
                feed_in = tf.reshape(inp, [-1, dim])            # e.g. feed_in = tf.reshape([None, 3, 3, 64], [576])
            else:
                feed_in, dim = (inp, input_shape[-1].value)     
            weights = self.make_var('weights', shape=[dim, num_out])   # e.g. shape = [3 * 3 * 64, 128]
            biases = self.make_var('biases', [num_out])                # e.g. shape = [128]
            op = tf.nn.relu_layer if relu else tf.nn.xw_plus_b        # define operation = ReLu or w*a + b
            fc = op(feed_in, weights, biases, name=name)
            return fc

    """
    Multi dimensional softmax,
    refer to https://github.com/tensorflow/tensorflow/issues/210
    compute softmax along the dimension of target
    the native softmax only supports batch_size x dimension
    """
    @layer
    def softmax(self, target, axis, name=None):
        max_axis = tf.reduce_max(target, axis, keepdims=True)
        target_exp = tf.exp(target-max_axis)
        normalize = tf.reduce_sum(target_exp, axis, keepdims=True)
        softmax = tf.div(target_exp, normalize, name)
        return softmax

In [4]:
class PNet(Network):  # Decorate with Network
    def setup(self):
        (self.feed('data') #pylint: disable=no-value-for-parameter, no-member
             .conv(3, 3, 10, 1, 1, padding='VALID', relu=False, name='conv1')
             .prelu(name='PReLU1')
             .max_pool(2, 2, 2, 2, name='pool1')
             .conv(3, 3, 16, 1, 1, padding='VALID', relu=False, name='conv2')
             .prelu(name='PReLU2')
             .conv(3, 3, 32, 1, 1, padding='VALID', relu=False, name='conv3')
             .prelu(name='PReLU3')
             .conv(1, 1, 2, 1, 1, relu=False, name='conv4-1')
             .softmax(3,name='prob1'))
        (self.feed('PReLU3') #pylint: disable=no-value-for-parameter
             .conv(1, 1, 4, 1, 1, relu=False, name='conv4-2'))
        
class RNet(Network):
    def setup(self):
        (self.feed('data') #pylint: disable=no-value-for-parameter, no-member
             .conv(3, 3, 28, 1, 1, padding='VALID', relu=False, name='conv1')
             .prelu(name='prelu1')
             .max_pool(3, 3, 2, 2, name='pool1')    # notice here is default padding = "SAME", thus why result [11,11, 28]
             .conv(3, 3, 48, 1, 1, padding='VALID', relu=False, name='conv2')
             .prelu(name='prelu2')
             .max_pool(3, 3, 2, 2, padding='VALID', name='pool2')
             .conv(2, 2, 64, 1, 1, padding='VALID', relu=False, name='conv3')
             .prelu(name='prelu3')
             .fc(128, relu=False, name='conv4')
             .prelu(name='prelu4')
             .fc(2, relu=False, name='conv5-1')
             .softmax(1,name='prob1'))

        (self.feed('prelu4') #pylint: disable=no-value-for-parameter
             .fc(4, relu=False, name='conv5-2'))
        
class ONet(Network):
    def setup(self):
        (self.feed('data') #pylint: disable=no-value-for-parameter, no-member
             .conv(3, 3, 32, 1, 1, padding='VALID', relu=False, name='conv1')
             .prelu(name='prelu1')
             .max_pool(3, 3, 2, 2, name='pool1')
             .conv(3, 3, 64, 1, 1, padding='VALID', relu=False, name='conv2')
             .prelu(name='prelu2')
             .max_pool(3, 3, 2, 2, padding='VALID', name='pool2')
             .conv(3, 3, 64, 1, 1, padding='VALID', relu=False, name='conv3')
             .prelu(name='prelu3')
             .max_pool(2, 2, 2, 2, name='pool3')
             .conv(2, 2, 128, 1, 1, padding='VALID', relu=False, name='conv4')
             .prelu(name='prelu4')
             .fc(256, relu=False, name='conv5')
             .prelu(name='prelu5')
             .fc(2, relu=False, name='conv6-1')
             .softmax(1, name='prob1'))

        (self.feed('prelu5') #pylint: disable=no-value-for-parameter
             .fc(4, relu=False, name='conv6-2'))

        (self.feed('prelu5') #pylint: disable=no-value-for-parameter
             .fc(10, relu=False, name='conv6-3'))

In [5]:
# function [boundingbox] = bbreg(boundingbox,reg)
def bbreg(boundingbox,reg):
    """Calibrate bounding boxes"""
    if reg.shape[1]==1:
        reg = np.reshape(reg, (reg.shape[2], reg.shape[3]))

    w = boundingbox[:,2]-boundingbox[:,0]+1
    h = boundingbox[:,3]-boundingbox[:,1]+1
    b1 = boundingbox[:,0]+reg[:,0]*w
    b2 = boundingbox[:,1]+reg[:,1]*h
    b3 = boundingbox[:,2]+reg[:,2]*w
    b4 = boundingbox[:,3]+reg[:,3]*h
    boundingbox[:,0:4] = np.transpose(np.vstack([b1, b2, b3, b4 ]))
    return boundingbox
 
def generateBoundingBox(imap, reg, scale, t):
    """Use heatmap to generate bounding boxes    
    """
    stride=2                       # because max pooling stride = 2
    cellsize=12

    imap = np.transpose(imap)      # 輸出結果將為[num_w_slide, num_h_slide], 相當於pyramid中每張圖片sliding window後的結果
    dx1 = np.transpose(reg[:,:,0]) # x1 for each bounding_box in imap, (shape = [num_w_slide, num_h_slide])
    dy1 = np.transpose(reg[:,:,1]) # y1 for each bounding_box in imap, (shape = [num_w_slide, num_h_slide])
    dx2 = np.transpose(reg[:,:,2]) # x2 for each bounding_box in imap, (shape = [num_w_slide, num_h_slide])
    dy2 = np.transpose(reg[:,:,3]) # y2 for each bounding_box in imap, (shape = [num_w_slide, num_h_slide])
    y, x = np.where(imap >= t)     # index mask for face_classification[num_w_slide, num_h_slide], see which one >= threshold[0]
                                   # we can know which sliding window >= threshold[0], by coordinate (num_w_slide=y, num_h_slide=x)
                                   # e.g. num_w_slide = y = [108 149 149 180], num_h_slide = x = [125 161 162 229]
    
    if y.shape[0]==1:             # if only 1 window was detected >= threshold[0]
        dx1 = np.flipud(dx1)      # why flip over ?? when only 1 detected??
        dy1 = np.flipud(dy1)      # why flip over ?? when only 1 detected??
        dx2 = np.flipud(dx2)      # why flip over ?? when only 1 detected??
        dy2 = np.flipud(dy2)      # why flip over ?? when only 1 detected??
    score = imap[(y,x)]
    reg = np.transpose(np.vstack([ dx1[(y,x)], dy1[(y,x)], dx2[(y,x)], dy2[(y,x)] ])) # vstack ensure matrix formate, final shape = [(num of > threshold), 4]
    
    if reg.size==0:               # if no bounding box was found
        reg = np.empty((0,3))
    bb = np.transpose(np.vstack([y,x]))          # bb = bounding box index
    q1 = np.fix((stride*bb+1)/scale)             # q1 = turn window back to original pic by index
    q2 = np.fix((stride*bb+cellsize-1+1)/scale) # q2 = turn window back to original pic by q1 + cellsize
    boundingbox = np.hstack([q1, q2, np.expand_dims(score,1), reg])
    return boundingbox, reg
 
# function pick = nms(boxes,threshold,type)
def nms(boxes, threshold, method):         # non-max suppression
    if boxes.size==0:
        return np.empty((0,3))
    x1 = boxes[:,0] # box coordinate x1
    y1 = boxes[:,1] # box coordinate y1
    x2 = boxes[:,2] # box coordinate x2
    y2 = boxes[:,3] # box coordinate y2
    s = boxes[:,4]  # box score
    area = (x2-x1+1) * (y2-y1+1)            # calculate box area
    I = np.argsort(s)                       # return the index from min to max
    pick = np.zeros_like(s, dtype=np.int16) # zeros matrix with same dimension in s.shape
    counter = 0
    while I.size>0:
        i = I[-1]                           # reverse direction, which means from max to min.
        pick[counter] = i                   # set pick[current_max_score_index, current_max_score_index ...] 
        counter += 1
        idx = I[0:-1]                       # the rest of boxes, except of max score one
        xx1 = np.maximum(x1[i], x1[idx])    # get intersection x1, np.maximum(current max ,rest boxes)
        yy1 = np.maximum(y1[i], y1[idx])    # get intersection y1
        xx2 = np.minimum(x2[i], x2[idx])    # get intersection x2
        yy2 = np.minimum(y2[i], y2[idx])    # get intersection y2
        w = np.maximum(0.0, xx2-xx1+1)      # intersection w
        h = np.maximum(0.0, yy2-yy1+1)      # intersection h
        inter = w * h                       # intersection area
        if method is 'Min':
            o = inter / np.minimum(area[i], area[idx])
        else:
            o = inter / (area[i] + area[idx] - inter)
        I = I[np.where(o<=threshold)]       # under current max score, delet IoU > threshold
    pick = pick[0:counter]
    return pick

# function [dy edy dx edx y ey x ex tmpw tmph] = pad(total_boxes,w,h)
def pad(total_boxes, w, h):
    """Compute the padding coordinates (pad the bounding boxes to square)"""
    tmpw = (total_boxes[:,2]-total_boxes[:,0]+1).astype(np.int32)
    tmph = (total_boxes[:,3]-total_boxes[:,1]+1).astype(np.int32)
    numbox = total_boxes.shape[0]

    dx = np.ones((numbox), dtype=np.int32)
    dy = np.ones((numbox), dtype=np.int32)
    edx = tmpw.copy().astype(np.int32)
    edy = tmph.copy().astype(np.int32)

    x = total_boxes[:,0].copy().astype(np.int32)   # bb x1 coord. (w-dir.)
    y = total_boxes[:,1].copy().astype(np.int32)   # bb y1 coord. (h-dir.)
    ex = total_boxes[:,2].copy().astype(np.int32)  # bb x2 coord. (w-dir.)
    ey = total_boxes[:,3].copy().astype(np.int32)  # bb y2 coord. (h-dir.)

    tmp = np.where(ex>w) # index of those x2 coordinate > window
    edx.flat[tmp] = np.expand_dims(-ex[tmp]+w+tmpw[tmp],1)
    ex[tmp] = w
    
    tmp = np.where(ey>h) # index of those y2 coordinate > window
    edy.flat[tmp] = np.expand_dims(-ey[tmp]+h+tmph[tmp],1)
    ey[tmp] = h

    tmp = np.where(x<1)  # index of those x1 coordinate < 1
    dx.flat[tmp] = np.expand_dims(2-x[tmp],1)
    x[tmp] = 1

    tmp = np.where(y<1)  # index of those y1 coordinate < 1
    dy.flat[tmp] = np.expand_dims(2-y[tmp],1)
    y[tmp] = 1
    # tmp[dy[k]-1:edy[k],dx[k]-1:edx[k],:] = img[y[k]-1:ey[k],x[k]-1:ex[k],:]
    # bb coord. in target (dy,edy,dx,edx), bb coord. in source（y,ey,x,ex）
    # set a suitable pixel corresponding to original img
    return dy, edy, dx, edx, y, ey, x, ex, tmpw, tmph

# function [bboxA] = rerec(bboxA)
def rerec(bboxA):
    """Convert bboxA to square."""
    # re-rectangular box
    h = bboxA[:,3]-bboxA[:,1]
    w = bboxA[:,2]-bboxA[:,0]
    l = np.maximum(w, h)
    bboxA[:,0] = bboxA[:,0]+w*0.5-l*0.5 # try to expand the shorter edge to rectangular
    bboxA[:,1] = bboxA[:,1]+h*0.5-l*0.5 # try to expand the shorter edge to rectangular
    bboxA[:,2:4] = bboxA[:,0:2] + np.transpose(np.tile(l,(2,1)))
    return bboxA

def imresample(img, sz):
    im_data = cv2.resize(img, (sz[1], sz[0]), interpolation=cv2.INTER_AREA) #@UndefinedVariable
            # 5-way to interpolation in cv2 : INTER_NEAREST、INTER_LINEAR、INTER_AREA、INTER_CUBICI、NTER_LANCZOS4
            # cv2.resize (h, w) 是相反的
    return im_data

In [6]:
def create_mtcnn(sess, model_path):
    if not model_path:
        model_path = os.getcwd()

    np.load.__defaults__=(None, True, True, 'latin1')   # declare load.__defaults__

    with tf.variable_scope('pnet'):
        data = tf.placeholder(tf.float32, (None,None,None,3), 'input')
        pnet = PNet({'data':data})  # define object pnet is PNet class
        #pnet.load(os.path.join(model_path, 'det1.npy'), sess)
        src_path = os.getcwd()
        model_path = os.path.join(src_path, args.model)
        pnet.load(os.path.join(model_path, 'det1.npy'), sess)
        
    with tf.variable_scope('rnet'):
        data = tf.placeholder(tf.float32, (None,24,24,3), 'input')
        rnet = RNet({'data':data})
        src_path = os.getcwd()
        model_path = os.path.join(src_path, args.model)
        rnet.load(os.path.join(model_path, 'det2.npy'), sess)
    with tf.variable_scope('onet'):
        data = tf.placeholder(tf.float32, (None,48,48,3), 'input')
        onet = ONet({'data':data})
        src_path = os.getcwd()
        model_path = os.path.join(src_path, args.model)
        onet.load(os.path.join(model_path, 'det3.npy'), sess)
            
    np.load.__defaults__=(None, False, True, 'ASCII')  # load.__defaults__  go back to original default
    
    pnet_fun = lambda img : sess.run(('pnet/conv4-2/BiasAdd:0', 'pnet/prob1:0'), feed_dict={'pnet/input:0':img})
    rnet_fun = lambda img : sess.run(('rnet/conv5-2/conv5-2:0', 'rnet/prob1:0'), feed_dict={'rnet/input:0':img})
    onet_fun = lambda img : sess.run(('onet/conv6-2/conv6-2:0', 'onet/conv6-3/conv6-3:0', 'onet/prob1:0'), feed_dict={'onet/input:0':img})
    return pnet_fun, rnet_fun, onet_fun

In [7]:
def detect_face(img, minsize, pnet, rnet, onet, threshold, factor):
    """Detects faces in an image, and returns bounding boxes and points for them.
    img: input image
    minsize: minimum faces' size
    pnet, rnet, onet: caffemodel
    threshold: threshold=[th1, th2, th3], th1-3 are three steps's threshold
    factor: the factor used to create a scaling pyramid of face sizes to detect in the image.
    """
    factor_count=0
    total_boxes=np.empty((0,9))
    points=np.empty(0)
    h=img.shape[0]        # e.g. h = 1500
    w=img.shape[1]        # e.g. w = 989
    minl=np.amin([h, w])  # minl = np.amin([h, w]) = 120 (get min. edge length)
    m=12.0/minsize        # m = 12/20, 因為PNet是用12x12訓練, 所以如果要求的minsizee!=12, 則需縮放到12x才可使用PNet參數
    minl=minl*m           # minl = 120 * (12/20)

    # create scale list for pyramid 建立金字塔所需scale list
    scales=[]
    while minl>=12:             # while PNet inpout > 12, e.g. minl = 120*(12/20), 120*(12/20)*f, 120*(12/20)*f^2, ...
        scales += [m*np.power(factor, factor_count)]  # scale += m * (f^(0,1,2,...)), till minl < 12, e.g. 12/20 *f^0, 12/20 *(f^0 + f^1),
        minl = minl*factor                            # minl = minl * f, e.g. 120 * (12/20) * f, 120 * (12/20) * f^2, 
        factor_count += 1                             # factor_count = 0, 1, 2, 3... till minl < 12
    
    # first stage                                      # P-Net processing
    for scale in scales:                              # e.g. 12/20 *f^0, 12/20 *(f^0 + f^1), ...
        hs=int(np.ceil(h*scale))                      # sample h 依scale list逐次縮小到minsize
        ws=int(np.ceil(w*scale))                      # sample w 依scale list逐次縮小到minsize
        im_data = imresample(img, (hs, ws))           # im_data = 依scale list 逐次resample縮放到目標sample尺寸
        im_data = (im_data-127.5)*0.0078125           # mean-zero & normalize
        img_x = np.expand_dims(im_data, 0)            # expand dimension axis=0, [h_x, w_x, c_x] -> [None, h_x, w_x, c_x]
        img_y = np.transpose(img_x, (0,2,1,3))        # transpose, [None, h_x, w_x, c_x] -> [None, w_x, h_x, c_x]
        out = pnet(img_y)                             # PNet_out = [bounding_box, face_classification]
        out0 = np.transpose(out[0], (0,2,1,3))        # bounding_box result, 輸出結果為[1, n_h_slide, n_w_slide, 4], n 即input image尺寸以12x12 作sliding window的數量
        out1 = np.transpose(out[1], (0,2,1,3))        # face_classification result, 輸出結果為[1, n_h_slide, n_w_slide, 2], n 即input image尺寸以12x12 作sliding window的數量
                                                      # [0] for probability of no face, [1]for probability of get face, [0] + [1] = 1
        boxes, _ = generateBoundingBox(out1[0,:,:,1].copy(), out0[0,:,:,:].copy(), scale, threshold[0])
                                                      # boxes = [None, 9]
        # inter-scale nms
        pick = nms(boxes.copy(), 0.5, 'Union')        # nms(boxes,threshold = 0.5,type = "Union")
        if boxes.size>0 and pick.size>0:
            boxes = boxes[pick,:]
            total_boxes = np.append(total_boxes, boxes, axis=0)

    numbox = total_boxes.shape[0]
    if numbox>0:
        pick = nms(total_boxes.copy(), 0.7, 'Union')   # nms(boxes,threshold = 0.7,type = "Union")
        total_boxes = total_boxes[pick,:]              # shape = [None, 9]
        regw = total_boxes[:,2]-total_boxes[:,0]       # sliding_window_width = q2(y) - q1(y)
        regh = total_boxes[:,3]-total_boxes[:,1]       # sliding_window_height = q2(x) - q1(x)
        qq1 = total_boxes[:,0]+total_boxes[:,5]*regw   # sliding window pixel_num + x1*width
        qq2 = total_boxes[:,1]+total_boxes[:,6]*regh   # sliding window pixel_num + y1*height
        qq3 = total_boxes[:,2]+total_boxes[:,7]*regw   # sliding window pixel_num + x2*width
        qq4 = total_boxes[:,3]+total_boxes[:,8]*regh   # sliding window pixel_num + y2*height
        total_boxes = np.transpose(np.vstack([qq1, qq2, qq3, qq4, total_boxes[:,4]])) # [qq1, qq2, qq3, qq4, score]
        total_boxes = rerec(total_boxes.copy())        # re-rectangular box
        total_boxes[:,0:4] = np.fix(total_boxes[:,0:4]).astype(np.int32)
        dy, edy, dx, edx, y, ey, x, ex, tmpw, tmph = pad(total_boxes.copy(), w, h)
                                                     # pad the bounding boxes to square), then we can send the box into RNet

    numbox = total_boxes.shape[0]
    if numbox>0:
        # second stage
        tempimg = np.zeros((24,24,3,numbox))
        for k in range(0,numbox):
            tmp = np.zeros((int(tmph[k]),int(tmpw[k]),3))
            tmp[dy[k]-1:edy[k],dx[k]-1:edx[k],:] = img[y[k]-1:ey[k],x[k]-1:ex[k],:] # set a suitable pixel corresponding to original img
            if tmp.shape[0]>0 and tmp.shape[1]>0 or tmp.shape[0]==0 and tmp.shape[1]==0:
                tempimg[:,:,:,k] = imresample(tmp, (24, 24))  # resample into 24x24x3 before into RNet
            else:
                return np.empty()
        tempimg = (tempimg-127.5)*0.0078125            # mean-zero & normalize
        tempimg1 = np.transpose(tempimg, (3,1,0,2))
        out = rnet(tempimg1)                           # RNet_out = [bounding_box result, face_classification result]
        out0 = np.transpose(out[0])                    # bounding_box result, 輸出結果為[4, None]
        out1 = np.transpose(out[1])                    # face_classification result, 輸出結果為[2, None]
                                                       # [0] for probability of no face, [1]for probability of get face, [0] + [1] = 1
        score = out1[1,:]
        ipass = np.where(score>threshold[1])
        total_boxes = np.hstack([total_boxes[ipass[0],0:4].copy(), np.expand_dims(score[ipass].copy(),1)])
        mv = out0[:,ipass[0]]
        if total_boxes.shape[0]>0:
            pick = nms(total_boxes, 0.7, 'Union')      # nms(boxes,threshold = 0.7,type = "Union")
            total_boxes = total_boxes[pick,:]
            total_boxes = bbreg(total_boxes.copy(), np.transpose(mv[:,pick]))
            total_boxes = rerec(total_boxes.copy())

    numbox = total_boxes.shape[0]
    if numbox>0:
        # third stage
        total_boxes = np.fix(total_boxes).astype(np.int32)
        dy, edy, dx, edx, y, ey, x, ex, tmpw, tmph = pad(total_boxes.copy(), w, h)
        tempimg = np.zeros((48,48,3,numbox))
        for k in range(0,numbox):
            tmp = np.zeros((int(tmph[k]),int(tmpw[k]),3))
            tmp[dy[k]-1:edy[k],dx[k]-1:edx[k],:] = img[y[k]-1:ey[k],x[k]-1:ex[k],:]
            if tmp.shape[0]>0 and tmp.shape[1]>0 or tmp.shape[0]==0 and tmp.shape[1]==0:
                tempimg[:,:,:,k] = imresample(tmp, (48, 48))
            else:
                return np.empty()
        tempimg = (tempimg-127.5)*0.0078125
        tempimg1 = np.transpose(tempimg, (3,1,0,2))
        out = onet(tempimg1)                           # ONet_out = [bounding_box result, facial landmark, face_classification result]
        out0 = np.transpose(out[0])                    # bounding_box result, 輸出結果為[4, None]
        out1 = np.transpose(out[1])                    # bounding_box result, 輸出結果為[10, None]
        out2 = np.transpose(out[2])                    # bounding_box result, 輸出結果為[2, None]
        score = out2[1,:]
        points = out1
        ipass = np.where(score>threshold[2])           
        points = points[:,ipass[0]]
        total_boxes = np.hstack([total_boxes[ipass[0],0:4].copy(), np.expand_dims(score[ipass].copy(),1)])
        mv = out0[:,ipass[0]]

        w = total_boxes[:,2]-total_boxes[:,0]+1
        h = total_boxes[:,3]-total_boxes[:,1]+1
        points[0:5,:] = np.tile(w,(5, 1))*points[0:5,:] + np.tile(total_boxes[:,0],(5, 1))-1
        points[5:10,:] = np.tile(h,(5, 1))*points[5:10,:] + np.tile(total_boxes[:,1],(5, 1))-1
        if total_boxes.shape[0]>0:
            total_boxes = bbreg(total_boxes.copy(), np.transpose(mv))
            pick = nms(total_boxes.copy(), 0.7, 'Min') # nms(boxes,threshold = 0.7,type = "Min")
            total_boxes = total_boxes[pick,:]
            points = points[:,pick]
                
    return total_boxes, points


In [8]:
"""Functions for building the face recognition network.

   facenet.py
   
"""
# MIT License
# 
# Copyright (c) 2016 David Sandberg
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# pylint: disable=missing-docstring
from subprocess import Popen, PIPE
from sklearn.model_selection import KFold
from scipy import interpolate
from tensorflow.python.training import training
import re
from tensorflow.python.platform import gfile
import math
from six import iteritems


def load_model(model, input_map=None):
    # Check if the model is a model directory (containing a metagraph and a checkpoint file)
    #  or if it is a protobuf file with a frozen graph
    src_path = os.getcwd()
    model_path = os.path.join(src_path, args.model) # download pre-trained facenet ResNet model, put under the "models" folder
    model_exp = os.path.expanduser(model_path)
    if (os.path.isfile(model_exp)):
        print('Model filename: %s' % model_exp)
        with gfile.FastGFile(model_exp,'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, input_map=input_map, name='')
    else:
        print('Model directory: %s' % model_exp)
        meta_file, ckpt_file = get_model_filenames(model_exp)
        
        print('Metagraph file: %s' % meta_file)
        print('Checkpoint file: %s' % ckpt_file)
      
        saver = tf.train.import_meta_graph(os.path.join(model_exp, meta_file), input_map=input_map)
        saver.restore(tf.get_default_session(), os.path.join(model_exp, ckpt_file))

def get_model_filenames(model_dir):
    files = os.listdir(model_dir)
    meta_files = [s for s in files if s.endswith('.meta')]
    if len(meta_files)==0:
        raise ValueError('No meta file found in the model directory (%s)' % model_dir)
    elif len(meta_files)>1:
        raise ValueError('There should not be more than one meta file in the model directory (%s)' % model_dir)
    meta_file = meta_files[0]
    ckpt = tf.train.get_checkpoint_state(model_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_file = os.path.basename(ckpt.model_checkpoint_path)
        return meta_file, ckpt_file

    meta_files = [s for s in files if '.ckpt' in s]
    max_step = -1
    for f in files:
        step_str = re.match(r'(^model-[\w\- ]+.ckpt-(\d+))', f)
        if step_str is not None and len(step_str.groups())>=2:
            step = int(step_str.groups()[1])
            if step > max_step:
                max_step = step
                ckpt_file = step_str.groups()[0]
    return meta_file, ckpt_file
        
def prewhiten(x):
    mean = np.mean(x)
    std = np.std(x)
    std_adj = np.maximum(std, 1.0/np.sqrt(x.size))
    y = np.multiply(np.subtract(x, mean), 1/std_adj)
    return y  


def get_dataset(path, has_class_directories=True):
    dataset = []
    path_exp = os.path.expanduser(path)
    classes = [path for path in os.listdir(path_exp) \
                    if os.path.isdir(os.path.join(path_exp, path))]
    classes.sort()
    nrof_classes = len(classes)
    for i in range(nrof_classes):
        class_name = classes[i]
        facedir = os.path.join(path_exp, class_name)
        image_paths = get_image_paths(facedir)
        dataset.append(ImageClass(class_name, image_paths))
  
    return dataset

def get_image_paths(facedir):
    image_paths = []
    if os.path.isdir(facedir):
        images = os.listdir(facedir)
        image_paths = [os.path.join(facedir,img) for img in images]
    return image_paths

def get_image_paths_and_labels(dataset):
    image_paths_flat = []
    labels_flat = []
    for i in range(len(dataset)):
        image_paths_flat += dataset[i].image_paths
        labels_flat += [i] * len(dataset[i].image_paths)
    return image_paths_flat, labels_flat
      
        
def load_data(image_paths, do_random_crop, do_random_flip, image_size, do_prewhiten=True):
    nrof_samples = len(image_paths)
    images = np.zeros((nrof_samples, image_size, image_size, 3))
    for i in range(nrof_samples):
        #img = misc.imread(image_paths[i])
        img = imageio.imread(image_paths[i])         # try this if you got error on 'scipy.misc' has no attribute 'imread'
        if img.ndim == 2:
            img = to_rgb(img)
        if do_prewhiten:
            img = prewhiten(img)
        img = crop(img, do_random_crop, image_size)
        img = flip(img, do_random_flip)
        images[i,:,:,:] = img
    return images

def crop(image, random_crop, image_size):
    if image.shape[1]>image_size:
        sz1 = int(image.shape[1]//2)
        sz2 = int(image_size//2)
        if random_crop:
            diff = sz1-sz2
            (h, v) = (np.random.randint(-diff, diff+1), np.random.randint(-diff, diff+1))
        else:
            (h, v) = (0,0)
        image = image[(sz1-sz2+v):(sz1+sz2+v),(sz1-sz2+h):(sz1+sz2+h),:]
    return image

def flip(image, random_flip):
    if random_flip and np.random.choice([True, False]):
        image = np.fliplr(image)
    return image

class ImageClass():
    "Stores the paths to images for a given class"
    def __init__(self, name, image_paths):
        self.name = name
        self.image_paths = image_paths
  
    def __str__(self):
        return self.name + ', ' + str(len(self.image_paths)) + ' images'
  
    def __len__(self):
        return len(self.image_paths)


In [9]:
import argparse                       # creat a parser to 從命令列直接讀取引數, 在jupyter運行時會因為sys.argv[1:]不為空而報錯

parser = argparse.ArgumentParser()

parser.add_argument('--mode', type=str, #choices=['TRAIN', 'CLASSIFY'],
    help='Indicates if a new classifier should be trained or a classification ' + 
    'model should be used for classification', default='TRAIN')  # default could be either 'TRAIN' or 'CLASSIFY' , choices=['TRAIN', 'CLASSIFY']
parser.add_argument('--data_dir', type=str, default=r"input_dir",
    help='Path to the data directory containing aligned LFW face patches.')
parser.add_argument('--model', type=str, default=r"models",
    help='Could be either a directory containing the meta_file and ckpt_file or a model protobuf (.pb) file')
parser.add_argument('--classifier_filename', default="my_classifier.pkl",
    help='Classifier model file name as a pickle (.pkl) file. ' + 
    'For training this is the output and for classification this is an input.')
parser.add_argument('--use_split_dataset', 
    help='Indicates that the dataset specified by data_dir should be split into a training and test set. ' +  
    'Otherwise a separate test set can be specified using the test_data_dir option.', action='store_true')
parser.add_argument('--test_data_dir', type=str,  default=r"input_dir",
    help='Path to the test data directory containing aligned images used for testing.')
parser.add_argument('--batch_size', type=int,
    help='Number of images to process in a batch.', default=90)
parser.add_argument('--image_size', type=int,
    help='Image size (height, width) in pixels.', default=160)
parser.add_argument('--seed', type=int,
    help='Random seed.', default=666)
parser.add_argument('--min_nrof_images_per_class', type=int,
    help='Only include classes with at least this number of images in the dataset', default=10)
parser.add_argument('--nrof_train_images_per_class', type=int,
    help='Use this number of images from each class for training and the rest for testing', default=5)
parser.add_argument('--debug', action='store_true',
                    help='Enable some debug outputs.')

args = parser.parse_args(args=["--mode", "CLASSIFY", "--data_dir", "input_dir", "--model", "models", "--classifier_filename", "my_classifier.pkl",
                               "--use_split_dataset", "--test_data_dir", r"input_dir", "--batch_size", "90", "--image_size", "160",
                               "--seed", "666", "--min_nrof_images_per_class", "10", "--nrof_train_images_per_class", "5",
                               "--debug"])

## Preliminary work before you run 前置作業
<br>Creat "models" folder, and put all medel parametes inside 創建 "models" 資料夾, 將所有參數放在此資料夾內
<br><p style="text-align:left;"><img src="images/dir_struc.png"  style="width:861px;height:496px;" align="left">

In [10]:
# coding=utf-8
"""Performs face detection in realtime. Based on code from https://github.com/shanren7/real_time_face_recognition"""
# MIT License
#
# Copyright (c) 2017 François Gervais
#
# This is the work of David Sandberg and shanren7 remodelled into a
# high level container. It's an attempt to simplify the use of such
# technology and provide an easy to use facial recognition package.
#
# https://github.com/davidsandberg/facenet
# https://github.com/shanren7/real_time_face_recognition
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

import pickle
import cv2
import time
#import align.detect_face
#import facenet


#gpu_memory_fraction = 0.3

src_path = os.getcwd()
model_path = os.path.join(src_path, args.model)
facenet_model_checkpoint = model_path

classifier_model_path = os.path.join(src_path, args.model)
classifier_model = classifier_model_path = os.path.join(classifier_model_path, args.classifier_filename)


debug = False


class Face:
    def __init__(self):
        self.name = None
        self.bounding_box = None
        self.image = None
        self.container_image = None
        self.embedding = None
        
class Recognition:
    def __init__(self):
        self.detect = Detection()
        self.encoder = Encoder()
        self.identifier = Identifier()

    def add_identity(self, image, person_name):
        faces = self.detect.find_faces(image)

        if len(faces) == 1:
            face = faces[0]
            face.name = person_name
            face.embedding = self.encoder.generate_embedding(face)
            return faces

    def identify(self, image):
        faces = self.detect.find_faces(image)

        for i, face in enumerate(faces):
            if debug:
                cv2.imshow("Face: " + str(i), face.image)
            face.embedding = self.encoder.generate_embedding(face)
            face.name = self.identifier.identify(face)

        return faces


class Identifier:
    def __init__(self):
        with open(classifier_model_path, 'rb') as infile:
            self.model, self.class_names = pickle.load(infile)

    def identify(self, face):
        if face.embedding is not None:
            predictions = self.model.predict_proba([face.embedding])
            best_class_indices = np.argmax(predictions, axis=1)
            return self.class_names[best_class_indices[0]]


class Encoder:
    def __init__(self):
        self.sess = tf.Session()
        with self.sess.as_default():
            load_model(facenet_model_checkpoint)

    def generate_embedding(self, face):
        # Get input and output tensors
        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

        #prewhiten_face = facenet.prewhiten(face.image)
        prewhiten_face = prewhiten(face.image)

        # Run forward pass to calculate embeddings
        feed_dict = {images_placeholder: [prewhiten_face], phase_train_placeholder: False}
        return self.sess.run(embeddings, feed_dict=feed_dict)[0]


class Detection:
    # face detection parameters
    minsize = 20  # minimum size of face
    threshold = [0.6, 0.7, 0.7]  # three steps's threshold
    factor = 0.709  # scale factor

    def __init__(self, face_crop_size=160, face_crop_margin=32):
        self.pnet, self.rnet, self.onet = self._setup_mtcnn()
        self.face_crop_size = face_crop_size
        self.face_crop_margin = face_crop_margin

    def _setup_mtcnn(self):
        with tf.Graph().as_default():
            #gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
            #sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
            sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
            with sess.as_default():
                #return align.detect_face.create_mtcnn(sess, None)
                return create_mtcnn(sess, None)

    def find_faces(self, image):
        faces = []

        bounding_boxes, _ = detect_face(image, self.minsize,
                                        self.pnet, self.rnet, self.onet,
                                        self.threshold, self.factor)
        for bb in bounding_boxes:
            face = Face()
            face.container_image = image
            face.bounding_box = np.zeros(4, dtype=np.int32)

            img_size = np.asarray(image.shape)[0:2]
            face.bounding_box[0] = np.maximum(bb[0] - self.face_crop_margin / 2, 0)
            face.bounding_box[1] = np.maximum(bb[1] - self.face_crop_margin / 2, 0)
            face.bounding_box[2] = np.minimum(bb[2] + self.face_crop_margin / 2, img_size[1])
            face.bounding_box[3] = np.minimum(bb[3] + self.face_crop_margin / 2, img_size[0])
            cropped = image[face.bounding_box[1]:face.bounding_box[3], face.bounding_box[0]:face.bounding_box[2], :]
            #face.image = misc.imresize(cropped, (self.face_crop_size, self.face_crop_size), interp='bilinear')
            face.image = resize(cropped, output_shape=(self.face_crop_size,self.face_crop_size))  # try this if you got error on 'scipy.misc' has no attribute 'imresize', scipy.misc.imre

            faces.append(face)

        return faces
    
def add_overlays(frame, faces, frame_rate):
    if faces is not None:
        for face in faces:
            face_bb = face.bounding_box.astype(int)
            cv2.rectangle(frame,
                          (face_bb[0], face_bb[1]), (face_bb[2], face_bb[3]),
                          (255, 0, 0), 2)
            if face.name is not None:
                cv2.putText(frame, face.name, (face_bb[0], face_bb[3]),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0),
                            thickness=2, lineType=2)

    cv2.putText(frame, str(frame_rate) + " fps", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0),
                thickness=2, lineType=2)


def main(args):
    frame_interval = 3  # Number of frames after which to run face detection
    fps_display_interval = 5  # seconds
    frame_rate = 0
    frame_count = 0

    video_capture = cv2.VideoCapture(0)
    #face_recognition = face.Recognition()
    face_recognition = Recognition()
    start_time = time.time()

    if args.debug:
        print("Debug enabled")
        #face.debug = True
        debug = True

    while(video_capture.isOpened()) :        # check ! (better usage than "while true")
        ret, frame = video_capture.read()    # capture frame-by-frame
        
        if (frame_count % frame_interval) == 0:
            faces = face_recognition.identify(frame)

            # Check our current fps
            end_time = time.time()
            if (end_time - start_time) > fps_display_interval:
                frame_rate = int(frame_count / (end_time - start_time))
                start_time = time.time()
                frame_count = 0

        add_overlays(frame, faces, frame_rate)

        frame_count += 1
        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break            
            
    # When everything is done, release the capture
    video_capture.release()
    cv2.destroyAllWindows()



In [11]:
main(args) # initiate main function

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Model directory: C:\Users\user\face_recog\DGEs\FaceNet\step4_real-time_face_recog\models
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\user\face_recog\DGEs\FaceNet\step4_real-time_face_recog\models\model-20180402-114759.ckpt-275
Debug enabled


## Here is our final result
<br><p style="text-align:left;"><img src="images/result.png"  style="width:861px;height:496px;" align="left">